## Compiling functions with Cython

In [2]:
import pyximport

In [3]:
%load_ext Cython

In [4]:
def my_op(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [5]:
%%cython

def my_cop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [6]:
my_op

<function __main__.my_op(data)>

In [7]:
my_cop

<function _cython_magic_5f50c224c320b4c4af3f7ea75a5b42ca.my_cop>

In [8]:
%timeit my_op([1,2,3,4,5])

2.22 µs ± 8.66 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit my_cop([1,2,3,4,5])

1.32 µs ± 4.67 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
import numpy

In [11]:
numpy.array([1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [12]:
d = numpy.array([1,2,3,4,5])
( d ** 2 + 3 ) / 4.5

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [13]:
%timeit ( d ** 2 + 3 ) / 4.5

3.61 µs ± 42.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
bigdata = numpy.random.randn(1000000)
bigdata.dtype

dtype('float64')

In [15]:
%timeit my_op(bigdata.copy())

1.22 s ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit ( bigdata.copy() ** 2 + 3 ) / 4.5

5.31 ms ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit my_cop(bigdata.copy())

1.08 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%cython
cimport numpy as np

def my_cop(np.ndarray[np.float_t, ndim=1] data):
    cdef int i

    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [19]:
my_cop

<function _cython_magic_f177c48b24ccc246b5140bf16197bc32.my_cop>

In [20]:
%timeit my_cop(bigdata.copy())

6.51 ms ± 8.42 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Just-in-time optimization with numba

In [21]:
from numba import jit

In [22]:
@jit
def my_nbop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [23]:
my_nbop

CPUDispatcher(<function my_nbop at 0x2af7e2a8a160>)

In [24]:
%timeit my_nbop([1,2,3,4,5])

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
/panfs/roc/msisoft/anaconda/python3-2020.07-mamba/lib/python3.8/site-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'data' of function 'my_nbop'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-22-de065de12a6b>", line 2:
@jit
def my_nbop(data):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


18.5 µs ± 7.34 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit my_nbop(bigdata.copy())

3.83 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Expression evaluators with numexpr

In [26]:
from numexpr import evaluate

In [27]:
smalldata = numpy.array([1,2,3,4,5])

In [28]:
evaluate("(smalldata**2 + 3) / 4.5")

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [ ]:
%timeit evaluate("(bigdata**2 + 3) / 4.5")